Inspired by Deep Learning for Computer Vision with Python [Rosebrock]  
Chapter 15  
MiniVGGNet: CIFAR-10

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load the CIFAR-10 dataset of 60,000 images of 10 classes
# 50,000 for training and 10,000 for testing
((trainX, trainY), (testX, testY)) = tf.keras.datasets.cifar10.load_data()

# scale data to the range of [0, 1]
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
# build the model
width = 32
height = 32
depth = 3
classes = 10

model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(height, width, depth))) # input layer

# first CONV => RELU => CONV => RELU => POOL layer set
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization(axis=-1)) # channels last
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization(axis=-1)) # channels last
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

# second CONV => RELU => CONV => RELU => POOL layer set
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization(axis=-1)) # channels last
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization(axis=-1)) # channels last
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

# first (and only) set of FC => RELU layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

# softmax classifier
model.add(tf.keras.layers.Dense(classes))
model.add(tf.keras.layers.Activation('softmax'))
model.summary()

# initialize the gradient descent optimizer
sgd = tf.keras.optimizers.SGD(
    learning_rate=0.01,
    weight_decay=0.01 / 40,
    momentum=0.9,
    nesterov=True)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# train the model
EPOCHS = 40
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=EPOCHS, verbose=1)

In [ ]:
# evaluate the network
predictions = model.predict(testX, batch_size=64)
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
report = classification_report(
    y_true=testY.argmax(axis=1),
    y_pred=predictions.argmax(axis=1),
    target_names=label_names)
print(report)

# plot the training loss and accuracy
plt.style.use('ggplot')
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history['loss'], label='train_loss')
plt.plot(np.arange(0, EPOCHS), H.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, EPOCHS), H.history['accuracy'], label='train_acc')
plt.plot(np.arange(0, EPOCHS), H.history['val_accuracy'], label='val_acc')
plt.title('Training Loss and Accuracy')
plt.xlabel('Epoch #')
plt.ylabel('Loss/Accuracy')
plt.legend()
plt.show()